In [1]:
import tensorflow as tf
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import tf_py_environment
from tf_agents.environments import gym_wrapper
import tf_agents
from tf_agents.utils import common
from tf_agents.policies import random_tf_policy
from tf_agents.specs import tensor_spec
from tf_agents.policies import random_tf_policy
from tf_agents.specs import tensor_spec
from tf_agents.replay_buffers import py_uniform_replay_buffer
import numpy as np
import pandas as pd
import datetime
# import gymnasium as gym
import gym
import numpy as np
import random
import MetaTrader5 as mt5
from collections import deque

In [2]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
def get_data(currency,):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2015, 1, 1), datetime.datetime.now()))
    ohlc_data['mtime']=pd.to_datetime(ohlc_data['time'],unit='s')
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data
def splitdf(data,size):
    splitdata=np.array_split(data, size)
    return splitdata

In [4]:
def yeardata(using_data,year,fmonth,smonth,fdate='01',sdate='01'):
    using_data2 = using_data[using_data['mtime'].dt.year == year]
    finalusing=using_data2.reset_index()
    data = finalusing[(finalusing['mtime'] >= f'{year}-{fmonth}-{fdate}') & (finalusing['mtime'] < f'{year}-{smonth}-{sdate}')]
    data.reset_index(drop=True, inplace=True)

    data = data.drop('index', axis=1)
    return data

In [5]:
dat=get_data('GOLD')
dat=yeardata(dat,2018,'01','12','01','30')
dfs=splitdf(dat,2)
dataf=dfs[0]

df = dataf.iloc[:1000]

eval_data = dataf.iloc[1000:2000]

c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
df.drop(columns=['time'], inplace=True)
df.drop(columns=['tick_volume'], inplace=True)
df.drop(columns=['spread'], inplace=True)
df.drop(columns=['real_volume'], inplace=True)
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\2474045239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['time'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\2474045239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['tick_volume'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\2474045239.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=

,Open,High,Low,Close,mtime
0,1309.57,1310.10,1309.28,1309.45,2018-01-02 09:00:00
1,1309.49,1309.58,1308.76,1309.19,2018-01-02 09:15:00
2,1309.29,1309.59,1308.67,1309.27,2018-01-02 09:30:00
3,1309.28,1309.39,1308.83,1309.29,2018-01-02 09:45:00
4,1309.28,1309.38,1308.37,1308.99,2018-01-02 10:00:00
...,...,...,...,...,...
995,1335.08,1335.18,1332.95,1333.24,2018-01-17 08:45:00
996,1333.33,1334.10,1332.89,1333.66,2018-01-17 09:00:00
997,1333.65,1334.16,1333.45,1334.06,2018-01-17 09:15:00
998,1334.01,1335.02,1333.85,1333.94,2018-01-17 09:30:00


In [7]:
eval_data.drop(columns=['time'], inplace=True)
eval_data.drop(columns=['tick_volume'], inplace=True)
eval_data.drop(columns=['spread'], inplace=True)
eval_data.drop(columns=['real_volume'], inplace=True)
eval_data

C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\218666643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data.drop(columns=['time'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\218666643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data.drop(columns=['tick_volume'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17252\218666643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_

,Open,High,Low,Close,mtime
1000,1334.13,1335.47,1334.06,1335.37,2018-01-17 10:00:00
1001,1335.37,1336.07,1334.98,1336.07,2018-01-17 10:15:00
1002,1336.07,1336.12,1334.89,1335.35,2018-01-17 10:30:00
1003,1335.37,1335.91,1334.78,1335.21,2018-01-17 10:45:00
1004,1335.21,1335.77,1334.41,1335.40,2018-01-17 11:00:00
...,...,...,...,...,...
1995,1344.34,1344.48,1343.75,1343.85,2018-02-01 05:45:00
1996,1343.85,1345.08,1343.75,1345.05,2018-02-01 06:00:00
1997,1345.05,1345.10,1344.55,1344.75,2018-02-01 06:15:00
1998,1344.76,1344.81,1344.35,1344.48,2018-02-01 06:30:00


In [8]:
ohlc_data = df[['Open', 'High', 'Low', 'Close']].values


In [9]:

class finalv(gym.Env):
    def __init__(self, initial_balance, data):
        super(finalv, self).__init__()

        # Define the OHLC data
        self.ohlc_data = data[['Open', 'High', 'Low', 'Close']].values
        self.current_index = 0
        self.current_trade = 0
        self.current_balance = initial_balance
        self.sl=0
        self.tp=0
        self.total_profit = 0.0
        
        self.entry=0
        

        # Define the observation space (using close prices only for simplicity)
        self.observation_space = gym.spaces.Box(low=0, high=np.max(self.ohlc_data[:,1]-self.ohlc_data[:,2]), shape=(12,))
        self.action_space = gym.spaces.Discrete(3)
        

    def count_profit(self, tradetype, entry, price):
        pr = price - entry
        if tradetype == 1:
            return price-entry
        elif tradetype == -1:
            return entry-price

    def _process_trade(self, profit):
        self.current_balance += profit
        self.total_profit += profit
    
    
        
   

    def step(self, action):
        reward = 0
        if self.current_trade==0:
            if action == 0:
                reward = 0
            elif action == 1:
                self.current_trade = 1
                self.entry = self.ohlc_data[self.current_index][3]
               
                reward = 0
            elif action == 2:
                self.current_trade = -1
                self.entry = self.ohlc_data[self.current_index][3]
                
                reward = 0
        elif self.current_trade==1:
            if action == 0:
                reward = 0
            elif action == 1:
                self.current_trade =0
                self._process_trade(self.count_profit(1, self.entry, self.ohlc_data[self.current_index][3]))
                reward = self.count_profit(1, self.entry, self.ohlc_data[self.current_index][3])
            elif action == 2:
                reward = 0
        elif self.current_trade==-1:
            if action == 0:
                reward = 0
            elif action == 1:
                
                self.current_trade = 0
                self._process_trade(self.count_profit(-1, self.entry, self.ohlc_data[self.current_index][3]))
                reward = self.count_profit(-1, self.entry, self.ohlc_data[self.current_index][3])
                
            elif action == 2:
               reward=0
            
                
           
            

        done = None
        if self.current_index >= len(self.ohlc_data) - 1:
          done=True
        


        prev5_items=self.ohlc_data[self.current_index-4: self.current_index+1]
        prevcandle_spread=prev5_items[:,1]-prev5_items[:,2]
        prevcandle_bodysizes=prev5_items[:,3]-prev5_items[:,0]
        candletype= -1 if self.ohlc_data[self.current_index][3]-self.ohlc_data[self.current_index][0] < 0 else 1
        istrade=self.current_trade
        state=np.concatenate((prevcandle_spread,prevcandle_bodysizes))
        state=np.insert(state,0,istrade)
        state=np.insert(state,0,candletype)
        state=np.array(state)
        
        discount = 0.99  # Set your desired discount factor
        step_type = 0  # Set the appropriate value for step_type

        # Construct the time step as a dictionary
        time_step = {'step_type': step_type, 'reward': reward, 'discount': discount, 'observation': state}
        info = {}
        if not done:
          self.current_index += 1
        return state, reward, done, info

    def render(self, mode='human'):
        if mode == 'human':
            print(f"Total Cumulative Profit: {self.total_profit}")
        else:
            raise NotImplementedError("Unsupported rendering mode")

    def reset(self):
        self.current_index = 6
        self.current_trade = 0
        self.total_profit = 0.0
        self.current_balance = 0
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0])



In [10]:
envn =finalv(100,df)

In [11]:

train = gym_wrapper.GymWrapper(envn)
train_env = tf_py_environment.TFPyEnvironment(train)


In [ ]:

tf_env = train_env

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 10

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    # print (type(action))
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)

In [12]:
eval_envn =finalv(100,eval_data)
eval = gym_wrapper.GymWrapper(eval_envn)
eval_env = tf_py_environment.TFPyEnvironment(eval)



In [13]:
fc_layer_params = (64,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)


In [14]:
from tensorflow.keras.optimizers.legacy import Adam


In [15]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)
opt=Adam(1e-3)

train_step_counter = tf.Variable(0)

agent00 = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=opt,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent00.initialize()

In [16]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),train_env.action_spec())



In [17]:
def compute_avg_return(environment, policy, num_episodes):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]



In [ ]:
compute_avg_return(train_env,random_policy,3)

In [19]:
from tf_agents.drivers import dynamic_step_driver

In [18]:
replay_buffer_capacity = 1000*32 # same capacity as the TFUniformReplayBuffer

py_replay_buffer = py_uniform_replay_buffer.PyUniformReplayBuffer(
    capacity=replay_buffer_capacity,
    data_spec=tensor_spec.to_nest_array_spec(agent00.collect_data_spec))

In [21]:
replay_observer = [py_replay_buffer.add_batch]

collect_steps_per_iteration = 10
collect_op = dynamic_step_driver.DynamicStepDriver(
  train_env,
  agent00.collect_policy,
  observers=replay_observer,
  num_steps=collect_steps_per_iteration).run()

In [39]:
from tf_agents.networks import sequential

In [40]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [41]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent2 = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent2.initialize()

In [35]:
dataset = py_replay_buffer.as_dataset(
    sample_batch_size=120,
    num_steps=2)


iterator = iter(dataset)

In [43]:
agent00.train = common.function(agent2.train)
agent00.train_step_counter.assign(0)
Btrain_eval=6
avg_return = compute_avg_return(eval_env, agent00.policy,Btrain_eval)
returns = [avg_return]
time_step = train_env.reset()
num_train_steps = 200
eval_interval=5
log_interval=15

for _ in range(num_train_steps):
  
  trajectories= next(iterator)
  loss = agent00.train(experience=trajectories).loss
  step = agent00.train_step_counter.numpy()
  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step,loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent00.policy, Btrain_eval)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)
  print(f"Step: {_}, Loss: {loss}")

step = 0: loss = 0.08811702579259872
step = 0: Average Return = -1.9500035047531128
Step: 0, Loss: 0.08811702579259872
step = 0: loss = 0.06029023602604866
step = 0: Average Return = -1.9500035047531128
Step: 1, Loss: 0.06029023602604866
step = 0: loss = 0.06028095260262489
step = 0: Average Return = -1.9500035047531128
Step: 2, Loss: 0.06028095260262489
step = 0: loss = 0.032077908515930176
step = 0: Average Return = -1.9500035047531128
Step: 3, Loss: 0.032077908515930176
step = 0: loss = 0.043895941227674484
step = 0: Average Return = -1.9500035047531128
Step: 4, Loss: 0.043895941227674484
step = 0: loss = 0.02418225072324276
step = 0: Average Return = -1.9500035047531128
Step: 5, Loss: 0.02418225072324276
step = 0: loss = 0.02702435478568077
